## 对wiki中文词库数据的导出
- 目标：防止内存过度使用
- 导出数据包括：
  - zh_wiki_id：id表示的中文单词wiki文章，一行
  - word_to_id.pkl：中文单词到id的映射词典
  - id_to_word.pkl：id到中文单词的映射词典
  - word_count.pkl：中文单词的出现次数的词典

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tqdm import tqdm

from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with open(filename) as f:
        data = f.readlines()
    return data

In [3]:
data = read_data("./data/zh_wiki_word_w_d")
vocabulary = []
for line in tqdm(data):
    line = line.strip()
    for item in line.split():
#         print(item)
        vocabulary.append(item)
#     break
# vocabulary = [v for v in vocabulary]
print('Data size', len(vocabulary))

100%|██████████| 4056734/4056734 [00:32<00:00, 126585.63it/s]

Data size 222822535


In [4]:
print(data[-1])
del data
vocabulary[-1]

做 头发 是 中国 大陆 互联网 流行语 之一 ， 该 词汇 发源 于 D 年 D 月 D 日 中国 大陆 娱乐圈 一个 爆炸 新闻 — — 李小璐 出轨 事件 ， 当天 当 民众 都 在 关注 各大 卫视 跨年 晚会 时 ， 李小璐 在 W 家中 过夜 的 视频 火遍 中国 娱乐圈 ， 而 贾乃亮 当晚 直播 说 李小璐 去 做 头发 了 ， 所以 不 在家 。 “ 做 头发 ” 梗 就此 而 来 ， 成为 继 娱乐圈 明星 们 晚上 关门 聊 剧本 之后 的 又 一 出轨 隐喻 。



'。'

In [5]:
vocabulary[:10]

['数学', '是', '利用', '符号语言', '研究', '数量', '结构', '变化', '以及', '空间']

In [10]:
len(set(vocabulary))

2041017

In [11]:
word_cnt = collections.Counter(vocabulary).most_common(2041017)
word_cnt[-10:]

[('雷瓦乡', 1),
 ('星棱大', 1),
 ('玩网', 1),
 ('💧', 1),
 ('充浩是', 1),
 ('我鸡哥', 1),
 ('人狠话', 1),
 ('纹缘', 1),
 ('猥琐男', 1),
 ('普泛', 1)]

In [12]:
for index, (k,v) in enumerate(word_cnt):
    if v <= 25:
        print(index)
        break

191551


In [14]:
import pickle
with open("data/word_count_w_d.pkl", "wb") as f:
    pickle.dump(word_cnt, f)

In [28]:
look_num = -2740
word_cnt[:look_num][-5:]

[('鲍洪升', 5), ('赫利纳', 5), ('但泽号', 5), ('革共', 5), ('柯尚', 4)]

In [29]:
vocabulary_size = 504175
vocabulary_size = 510000 - 2740

def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [30]:
# Filling 4 global variables:
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary, vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 2297816], ('，', 14808459), ('的', 9437707), ('。', 8706775), ('D', 8592359)]
Sample data [1348, 9, 501, 237319, 141, 894, 5, 499, 5, 331] ['数学', '是', '利用', '符号语言', '研究', '数量', '、', '结构', '、', '变化']


In [ ]:
Most common words (+UNK) [['UNK', 2291206], ('的', 9437707), ('年', 2807078), ('在', 2679806), ('是', 2112111)]
Sample data [1296, 4, 462, 235572, 116, 850, 460, 301, 53, 611] ['数学', '是', '利用', '符号语言', '研究', '数量', '结构', '变化', '以及', '空间']

Most common words (+UNK) [['UNK', 2297816], ('，', 14808459), ('的', 9437707), ('。', 8706775), ('D', 8592359)]
Sample data [1348, 9, 501, 237319, 141, 894, 5, 499, 5, 331] ['数学', '是', '利用', '符号语言', '研究', '数量', '、', '结构', '、', '变化']

In [9]:
count[-10:]

[('郝琼', 5),
 ('线古坑', 5),
 ('尼乡', 5),
 ('泰什蒂乡', 5),
 ('梅斐', 5),
 ('卢德姆', 5),
 ('鲍洪升', 5),
 ('赫利纳', 5),
 ('但泽号', 5),
 ('革共', 5)]

In [32]:
with open("data/zh_wiki_id_w_d", "w") as f:
    context = [str(i) for i in data]
    f.write(" ".join(context))

In [34]:
import pickle
with open("data/word_to_id_w_d.pkl", "wb") as f:
    pickle.dump(dictionary, f)
with open("data/id_to_word_w_d.pkl", "wb") as f:
    pickle.dump(reverse_dictionary, f)
# with open("data/word_count_w_d.pkl", "wb") as f:
#     pickle.dump(count, f)

##  Load Data Test

In [1]:
import pickle

In [3]:
zh_wiki_id = open("data/zh_wiki_id").readline()
word_to_id = pickle.load(open("data/word_to_id.pkl", "rb"))
id_to_word = pickle.load(open("data/id_to_word.pkl", "rb"))
word_count = pickle.load(open("data/count.pkl", "rb"))

In [2]:
zh_wiki_id = open("data/zh_wiki_id_w_d").readline()
word_to_id = pickle.load(open("data/word_to_id_w_d.pkl", "rb"))
id_to_word = pickle.load(open("data/id_to_word_w_d.pkl", "rb"))
# word_count = pickle.load(open("data/count.pkl", "rb"))

In [4]:
len(word_to_id)

507260

In [5]:
def getWord(data, num, data_index):
    sub_data_string = data[data_index:data_index+num*(6+1)]
    print(sub_data_string)
    result = []
    for index, item in enumerate(sub_data_string.split()):
        if index == num: break
        data_index += len(item) + 1
        result.append(int(item))
    if len(result) < num:
        return getWord(data, num, 0)
    assert len(result) == num
    return result, data_index

In [27]:
len(zh_wiki_id)

727934301

In [49]:
getWord(zh_wiki_id, 1, 727934301-5)

21556


([21556], 727934302)

In [10]:
import numpy as np
import collections

In [8]:
def generate_batch(batch_size, skip_window, num_skips):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size, num_skips), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    assert batch_size >= span
    buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builtin
#     if data_index + span*(6+1) > len(data):
#         data_index = 0
    result, data_index = getWord(zh_wiki_id, span, data_index)
    buffer.extend(result)
#     data_index += span
    for i in range(batch_size):
#         print(data_index, data_index + span, buffer)
        context_words = [w for w in range(span) if w != skip_window]
#         words_to_use = random.sample(context_words, num_skips)
        # context tokens are just all the tokens in buffer except the target
#         batch[i, :] = [token for idx, token in enumerate(buffer) if idx != context_window]
#         batch[i, :] = [buffer[token] for idx, token in enumerate(words_to_use)]
        batch[i, :] = [buffer[token] for idx, token in enumerate(context_words)]
        labels[i, 0] = buffer[skip_window]
        result, data_index = getWord(zh_wiki_id, 1, data_index)
        buffer.append(result[0])
        if data_index > len(zh_wiki_id):
            result, data_index = getWord(zh_wiki_id, span-1, 0)
            buffer.extend(result)
        if i == batch_size - span:
            last_index = data_index
#         data_index = (data_index + 1) % len(data)
        
#         print("batch:", [i for i in batch], "\nlabels:", [ j for i in labels for j in i])
#         print("batch:", batch[i], "\nlabels:", labels[i])
    # Backtrack a little bit to avoid skipping words in the end of a batch
#     data_index = (data_index + len(data) - span) % len(data)
    data_index = last_index
#     print("batch:", batch, "\nlabels:", labels)
    return batch, labels

In [16]:
# data_index = 0

batch, labels = generate_batch(batch_size=20, skip_window=5, num_skips=2*5)
print(data_index)
print("batch：", batch.shape)
print("labels：", labels.shape)
for i in range(20):
    print(batch[i, 0], id_to_word[batch[i, 0]],
          batch[i, 1], id_to_word[batch[i, 1]],
          batch[i, 2], id_to_word[batch[i, 2]],
          batch[i, 3], id_to_word[batch[i, 3]],
          batch[i, 4], id_to_word[batch[i, 4]],
          batch[i, 5], id_to_word[batch[i, 5]],
          batch[i, 6], id_to_word[batch[i, 6]],
          batch[i, 7], id_to_word[batch[i, 7]],
          batch[i, 8], id_to_word[batch[i, 8]],
          batch[i, 9], id_to_word[batch[i, 9]],
          '->', labels[i, 0], id_to_word[labels[i, 0]])
# for i in range(8):
#     print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
#         reverse_dictionary[labels[i, 0]])

73 68 7400 2 8354 41 690 23 262 271 9578 10038 190 2 3132 3 607 1348 2 1908 1
10038 1
190 2 3
2 3132 
3132 3 
3 607 1
607 134
1348 2 
2 1908 
1908 18
18 1677
1677 34
3419 55
550 18 
18 680 
680 253
253 23 
23 1718
17185 2
2 17826
17826 3
318
batch： (20, 10)
labels： (20, 1)
73 以及 68 从 7400 选定 2 的 8354 公理 690 定义 23 中 262 建立 271 起 9578 严谨 -> 41 及
68 从 7400 选定 2 的 8354 公理 41 及 23 中 262 建立 271 起 9578 严谨 10038 推导 -> 690 定义
7400 选定 2 的 8354 公理 41 及 690 定义 262 建立 271 起 9578 严谨 10038 推导 190 出 -> 23 中
2 的 8354 公理 41 及 690 定义 23 中 271 起 9578 严谨 10038 推导 190 出 2 的 -> 262 建立
8354 公理 41 及 690 定义 23 中 262 建立 9578 严谨 10038 推导 190 出 2 的 3132 定理 -> 271 起
41 及 690 定义 23 中 262 建立 271 起 10038 推导 190 出 2 的 3132 定理 3 。 -> 9578 严谨
690 定义 23 中 262 建立 271 起 9578 严谨 190 出 2 的 3132 定理 3 。 607 基础 -> 10038 推导
23 中 262 建立 271 起 9578 严谨 10038 推导 2 的 3132 定理 3 。 607 基础 1348 数学 -> 190 出
262 建立 271 起 9578 严谨 10038 推导 190 出 3132 定理 3 。 607 基础 1348 数学 2 的 -> 2 的
271 起 9578 严谨 10038 推导 190 出 2 的 3 。 607 基础 1348 数学 2 的 1908

In [78]:
# data_index = 0

batch, labels = generate_batch(batch_size=20, skip_window=5, num_skips=2*5)
print(data_index)
print("batch：", batch.shape)
print("labels：", labels.shape)
for i in range(20):
    print(batch[i, 0], id_to_word[batch[i, 0]],
          batch[i, 1], id_to_word[batch[i, 1]],
          batch[i, 1], id_to_word[batch[i, 2]],
          batch[i, 1], id_to_word[batch[i, 3]],
          batch[i, 1], id_to_word[batch[i, 4]],
          batch[i, 1], id_to_word[batch[i, 5]],
          batch[i, 1], id_to_word[batch[i, 6]],
          batch[i, 1], id_to_word[batch[i, 7]],
          batch[i, 1], id_to_word[batch[i, 8]],
          batch[i, 1], id_to_word[batch[i, 9]],
          '->', labels[i, 0], id_to_word[labels[i, 0]])
# for i in range(8):
#     print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
#         reverse_dictionary[labels[i, 0]])

21 273 4340 294 6398 119 858 197 49216 77 1 11664 53 48 7268 1 8205 26 649 12
11664 5
53 48 7
48 7268
7268 1 
1 8205 
8205 26
26 649 
649 12 
12 235 
235 244
244 942
9420 98
9874 16
163 1 3
1 3057 
3057 56
567 129
1296 1 
1 1851 
1851 9 
249
batch： (20, 10)
labels： (20, 1)
21 而 273 产生 273 数学家 273 们 273 拓展 273 概念 273 为了 273 公式化 273 新 273 的 -> 119 这些
273 产生 4340 数学家 4340 们 4340 拓展 4340 这些 4340 为了 4340 公式化 4340 新 4340 的 4340 猜想 -> 858 概念
4340 数学家 294 们 294 拓展 294 这些 294 概念 294 公式化 294 新 294 的 294 猜想 294 以及 -> 197 为了
294 们 6398 拓展 6398 这些 6398 概念 6398 为了 6398 新 6398 的 6398 猜想 6398 以及 6398 从 -> 49216 公式化
6398 拓展 119 这些 119 概念 119 为了 119 公式化 119 的 119 猜想 119 以及 119 从 119 选定 -> 77 新
119 这些 858 概念 858 为了 858 公式化 858 新 858 猜想 858 以及 858 从 858 选定 858 的 -> 1 的
858 概念 197 为了 197 公式化 197 新 197 的 197 以及 197 从 197 选定 197 的 197 公理 -> 11664 猜想
197 为了 49216 公式化 49216 新 49216 的 49216 猜想 49216 从 49216 选定 49216 的 49216 公理 49216 及 -> 53 以及
49216 公式化 77 新 77 的 77 猜想 77 以及 77 选定 77 的 77 公理 77 及 77 定义 -> 48 从
